In [1]:
%matplotlib inline


# Extracting signals from a brain parcellation

Here we show how to extract signals from a brain parcellation and compute
a correlation matrix.

We also show the importance of defining good confounds signals: the
first correlation matrix is computed after regressing out simple
confounds signals: movement regressors, white matter and CSF signals, ...
The second one is without any confounds: all regions are connected to
each other.


One reference that discusses the importance of confounds is `Varoquaux and
Craddock, Learning and comparing functional connectomes across subjects,
NeuroImage 2013
<http://www.sciencedirect.com/science/article/pii/S1053811913003340>`_.

This is just a code example, see the `corresponding section in the
documentation <parcellation_time_series>` for more.

<div class="alert alert-info"><h4>Note</h4><p>This example needs SciPy >= 1.0.0 for the reordering of the matrix.</p></div>


In [2]:
import ast
import gzip
import io
import json
import nilearn
import os
import sklearn
import tarfile
import tempfile
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sns
from bids_validator import BIDSValidator

from fetch_difumo import fetch_difumo
from io import BytesIO
from pathlib import Path
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join as pjoin
from pandas import DataFrame as df
from tempfile import TemporaryDirectory as tmpdir
from tempfile import TemporaryFile as tmpfile
from tqdm import tqdm
from typing import Union
from collections.abc import Iterable
from typing import Sequence

import loadutils as lu
import sniffbytes as snif
import scanzip as szip
import shutil

from nilearn import masking
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, plot_epi
from nilearn.image import concat_imgs, mean_img
from nilearn.input_data import NiftiMasker
from cimaqprep import participant_data

/home/fnadeau/myenv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [4]:
# % install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
# % load_ext autotime
import warnings
warnings.filterwarnings('ignore')
from cimaqprep.participant_data import participant_data
subject00 = participant_data(cimaq_nov_dir = xpu('~/../../data/cisl/DATA/cimaq_20190901'),
                             cimaq_mar_dir = xpu('~/../../data/cisl/DATA/cimaq_03-19'),
                             events_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_events/events'),
                             behav_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_behavioural/behavioural'),
                             participants_path = xpu('~/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/participants/'))

100%|█████████████████████████████████████████| 120/120 [00:19<00:00,  6.17it/s]


## Retrieve the atlas and the data



In [4]:
from nilearn import datasets


dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm',
                                              data_dir=pjoin(dname(os.getcwd()),'harvard_oxford'))
difumo_dir=pjoin(dname(os.getcwd()),'DiFuMo_atlases')
difumo512_3mm=datasets.atlas.fetch_atlas_difumo(dimension=512,
                                                resolution_mm=3,
                                                data_dir=difumo_dir,
                                                resume=True,
                                                verbose=1)
difumo512_3mm.labels
# atlas_filename, labels = difumo512_3mm.maps,difumo512_3mm.maps[0],difumo512_3mm.maps,difumo512_3mm.labels
atlas_filename, labels = difumo512_3mm.maps, difumo512_3mm.labels
# # One subject fmri data
# # data=subject_data
# fmri_filenames=subject00.resampled_fmri_to_events
# data = datasets.fetch_development_fmri(n_subjects=1,
#                                        data_dir=pjoin(dname(os.getcwd()),'development_fmri'))
# fmri_filenames = data.func[0]

In [5]:
type(dataset)

sklearn.utils.Bunch

## Extract signals on a parcellation defined by labels
Using the NiftiLabelsMasker



#### Here we go from nifti files to the signal time series in a numpy array.
#### Note how we give confounds to be regressed out during signal extraction.

In [16]:
from nilearn.image import concat_imgs, mean_img
mean_img(concat_imgs(subject00.resampled_fmri_to_events.values))

(80, 80, 41)

NameError: name 'difumo' is not defined

In [76]:
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=mean_img(difumo512_3mm.maps),
                           labels=difumo512_3mm.labels,
                           standardize=True,
                           memory='nilearn_cache', verbose=5)


In [69]:
# help(masker.fit_transform)
help(NiftiLabelsMasker)

Help on class NiftiLabelsMasker in module nilearn.input_data.nifti_labels_masker:

class NiftiLabelsMasker(nilearn.input_data.base_masker.BaseMasker, nilearn._utils.cache_mixin.CacheMixin)
 |  NiftiLabelsMasker(labels_img, labels=None, background_label=0, mask_img=None, smoothing_fwhm=None, standardize=False, standardize_confounds=True, high_variance_confounds=False, detrend=False, low_pass=None, high_pass=None, t_r=None, dtype=None, resampling_target='data', memory=Memory(location=None), memory_level=1, verbose=0, strategy='mean', reports=True)
 |  
 |  Class for masking of Niimg-like objects.
 |  
 |  NiftiLabelsMasker is useful when data from non-overlapping volumes should
 |  be extracted (contrarily to NiftiMapsMasker). Use case: Summarize brain
 |  signals from clusters that were obtained by prior K-means or Ward
 |  clustering.
 |  
 |  Parameters
 |  ----------
 |  labels_img : Niimg-like object
 |      See http://nilearn.github.io/manipulating_images/input_output.html
 |      

In [77]:

time_series = masker.fit_transform(imgs=mean_img(concat_imgs(fmri_filenames.values)))
#     imgs=mean_img(concat_imgs(subject00.resampled_fmri_to_events.values)))
#                                    confounds=data.confounds)

[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image(
shape=(104, 123, 104),
affine=array([[   3.,    0.,    0.,  -96.],
       [   0.,    3.,    0., -132.],
       [   0.,    0.,    3.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
Resampling labels
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(<nibabel.nifti1.Nifti1Image object at 0x7f9213a55dd0>, <nilearn.input_data.nifti_labels_masker._ExtractionFunctor object at 0x7f922ac5a7d0>, 
{ 'background_label': 0,
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'high_variance_confounds': False,
  'labels': rec.array([(  1, 'Superior occipital sulcus inferior LH', 'DorsAttnB', 'DorsAttnA', 0.57588 , 0.346929, 0.077031),
           ...,
           (512, 'Precentral sulcus superior RH', 'DorsAttnB', 'DorsAttnB', 0.622743, 0.314315, 0.062787)],
          dtype=[('component', '<i8'), ('difumo_names'

In [57]:
time_series.shape

(1, 43670)

In [39]:
# subject00.resampled_confounds =
df([(item[0],tuple(val.mid for val in pd.cut(item[1],bins=subject00.events.shape[0])))
 for item in subject00.confounds.iteritems()])[1]

0     (-0.006619999999999999, -0.08829999999999999, ...
1     (0.09545000000000001, 0.0405, 0.02950000000000...
2     (0.1017, 0.0032613, -0.14100000000000001, 0.08...
3     (-0.3145, -0.14950000000000002, -0.02515, -0.3...
4     (-0.02975, -0.041999999999999996, -0.033850000...
5     (-0.1625, -0.125, -0.07815, -0.209, -0.2415, -...
6     (0.425, 0.36, 0.949, 0.20700000000000002, 0.27...
7     (0.0036650000000000003, 0.996, 0.996, 0.996, 0...
8     (1.395, 1.395, 1.395, 1.395, 1.395, 1.395, 1.3...
9     (1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, ...
10    (1.395, 1.395, 1.395, 1.395, 1.395, 1.395, 1.3...
11    (1.4, 1.4, 1.4, 1.4, 1.4, 1.3765, 1.353, 1.353...
12    (1.395, 1.395, 1.395, 1.395, 1.371500000000000...
13    (1.4, 1.4, 1.4, 1.3765, 1.353, 1.3295, 1.306, ...
14    (1.395, 1.395, 1.395, 1.3715000000000002, 1.34...
15    (1.4, 1.4, 1.4, 1.353, 1.3295, 1.2825, 1.212, ...
16    (1.395, 1.395, 1.3715000000000002, 1.348, 1.30...
17    (1.4, 1.4, 1.3765, 1.3295, 1.2825, 1.212, 

## Compute and display a correlation matrix
### - Plot the correlation matrix
#### - Make a large figure
#### - Mask the main diagonal for visualization:
#### - The labels we have start with the background (0).
    - Hence we skip the first label.
#### - Matrices are ordered for block-like representation


In [79]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([time_series])[0]

In [80]:
correlation_matrix.shape

(43670, 43670)

In [78]:


import numpy as np
from nilearn import plotting
np.fill_diagonal(correlation_matrix, 0)
plotting.plot_matrix(correlation_matrix,
                     figure=(10, 8),
                     labels=difumo512_3mm.labels,
                     vmax=0.8,
                     vmin=-0.8,
                     reorder=True)

ValueError: Length of labels unequal to length of matrix.

## Same thing without confounds, to stress the importance of confounds



In [ ]:
time_series = masker.fit_transform(fmri_filenames)
# Note how we did not specify confounds above. This is bad!

correlation_matrix = correlation_measure.fit_transform([time_series])[0]

# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)

plotting.plot_matrix(correlation_matrix, figure=(10, 8), labels=labels[1:],
                     vmax=0.8, vmin=-0.8, title='No confounds', reorder=True)

plotting.show()